## 1. 구글 드라이브에서 마운틴하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 2. 파일 경로 지정하기

/content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio.zip

## 3. 압축 해제하기

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio.zip" -d "/content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio"


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset/XRGRGSXY.ogg  
  inflating: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset/XRGRRNPJ.ogg  
  inflating: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset/XRGXYWHM.ogg  
  inflating: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset/XRHCAMKB.ogg  
  inflating: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset/XRHLBJQP.ogg  
  inflating: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset/XRHRBDIM.ogg  
  inflating: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset/XRHUGFFU.ogg  
  inflating: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset/XRIBMZZI.ogg  
  in

## 4. csv파일을 읽어 파일 나누기


In [3]:
import pandas as pd
import os
import shutil

# CSV 파일 경로
csv_path = '/content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_path)

# 원본 오디오 파일이 저장된 경로
audio_base_path = '/content/drive/MyDrive/Colab Notebooks/wav2vec classification/Fake Audio/Fake Audio/dataset'

# real과 fake 폴더 경로
real_dir = '/content/drive/MyDrive/Colab Notebooks/wav2vec classification/real'
fake_dir = '/content/drive/MyDrive/Colab Notebooks/wav2vec classification/fake'

# real과 fake 폴더가 없다면 생성
os.makedirs(real_dir, exist_ok=True)
os.makedirs(fake_dir, exist_ok=True)

# 각 id에 따라 파일을 이동
for index, row in df.iterrows():
    file_name = row['id'] + '.ogg'  # 파일명은 id와 확장자 조합
    label = row['label']             # 레이블은 'real' 또는 'fake'

    # 원본 파일 경로
    src_path = os.path.join(audio_base_path, file_name)

    # 이동할 대상 경로 설정
    if label == 'real':
        dest_path = os.path.join(real_dir, file_name)
    else:
        dest_path = os.path.join(fake_dir, file_name)

    # 파일 이동
    if os.path.exists(src_path):
        shutil.move(src_path, dest_path)
    else:
        print(f"파일 {src_path}이(가) 존재하지 않습니다.")
